In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python import keras as kt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [2]:

X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")


### Shape of data

In [3]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 25, 1000)
Test data shape: (443, 25, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [13]:
# prep datasets for RNN

# fomat data to (batch_size, timesteps, input_dim)
k_X_train = np.swapaxes(X_train_valid, 1, 2)
k_X_test = np.swapaxes(X_test, 1, 2)
print("Swapped axes:",k_X_train.shape, k_X_test.shape)

# remove VOG channels (23-25)
k_X_train = k_X_train[: , : , :22 ]
k_X_test = k_X_test[: , : , :22 ]
print("Removed VOG channels:", k_X_train.shape, k_X_test.shape)

# encode output labels
print("Raw labels:", y_train_valid[0:10])
k_y_train = y_train_valid - 769
k_y_test = y_test - 769
print("Fixed:", k_y_train[0:10])
k_y_train_categ = to_categorical(k_y_train, 4)
k_y_test_categ = to_categorical(k_y_test, 4)
print("Categorical one-hot encoding:\n",k_y_train_categ[0:3])

Swapped axes: (2115, 1000, 25) (443, 1000, 25)
Removed VOG channels: (2115, 1000, 22) (443, 1000, 22)
Raw labels: [771 772 769 769 769 769 771 770 772 772]
Fixed: [2 3 0 0 0 0 2 1 3 3]
Categorical one-hot encoding:
 [[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]


In [14]:
# RNN imports

import keras
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, RNN, SimpleRNN
from keras.utils import to_categorical

In [21]:
# RNN model parameters
k_hidden_size = 100
k_input_shape = k_X_train.shape[1:]

# RNN model
model = Sequential()
model.add(SimpleRNN(k_hidden_size, input_shape=k_input_shape))
model.add(Dense(4, activation='softmax'))

model.compile(loss = 'categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'],
             )


model.fit(k_X_train, k_y_train_categ, epochs=100)

Epoch 1/100
2115/2115 [==============================] - 9s 4ms/step - loss: 1.5185 - acc: 0.2567
Epoch 2/100
2115/2115 [==============================] - 8s 4ms/step - loss: 1.4008 - acc: 0.3031
Epoch 3/100
2115/2115 [==============================] - 9s 4ms/step - loss: 1.3561 - acc: 0.3400
Epoch 4/100
2115/2115 [==============================] - 9s 4ms/step - loss: 1.3301 - acc: 0.3485
Epoch 5/100
2115/2115 [==============================] - 9s 4ms/step - loss: 1.2974 - acc: 0.3872
Epoch 6/100
2115/2115 [==============================] - 9s 4ms/step - loss: 1.2772 - acc: 0.4014
Epoch 7/100
2115/2115 [==============================] - 9s 4ms/step - loss: 1.2561 - acc: 0.4255
Epoch 8/100
2115/2115 [==============================] - 9s 4ms/step - loss: 1.2376 - acc: 0.4473
Epoch 9/100
2115/2115 [==============================] - 9s 4ms/step - loss: 1.2275 - acc: 0.4515
Epoch 10/100
2115/2115 [==============================] - 9s 4ms/step - loss: 1.2101 - acc: 0.4558
Epoch 11/100
2115/2

2115/2115 [==============================] - 12s 6ms/step - loss: 0.3786 - acc: 0.8747
Epoch 84/100
2115/2115 [==============================] - 10s 5ms/step - loss: 0.3554 - acc: 0.8946
Epoch 85/100
2115/2115 [==============================] - 10s 5ms/step - loss: 0.3502 - acc: 0.8875
Epoch 86/100
2115/2115 [==============================] - 11s 5ms/step - loss: 0.3514 - acc: 0.8922
Epoch 87/100
2115/2115 [==============================] - 12s 6ms/step - loss: 0.3864 - acc: 0.8747
Epoch 88/100
2115/2115 [==============================] - 11s 5ms/step - loss: 0.3510 - acc: 0.8917
Epoch 89/100
2115/2115 [==============================] - 10s 5ms/step - loss: 0.3327 - acc: 0.9012
Epoch 90/100
2115/2115 [==============================] - 10s 5ms/step - loss: 0.3242 - acc: 0.9026
Epoch 91/100
2115/2115 [==============================] - 9s 4ms/step - loss: 0.3331 - acc: 0.8960
Epoch 92/100
2115/2115 [==============================] - 11s 5ms/step - loss: 0.3319 - acc: 0.8955
Epoch 93/100
2

In [22]:

# evaluate

model.evaluate(k_X_test, k_y_test_categ, verbose=1)

443/443 [==============================] - 1s 2ms/step


[2.503969310906886, 0.27088036043380237]

# That is disappointing